In [131]:
!wget https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv

--2023-08-03 15:14:05--  https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115525 (113K) [text/plain]
Saving to: ‘NSE-TATAGLOBAL.csv.1’

NSE-TATAGLOBAL.csv. 100%[===================>] 112.82K  --.-KB/s    in 0.02s   

2023-08-03 15:14:05 (5.30 MB/s) - ‘NSE-TATAGLOBAL.csv.1’ saved [115525/115525]



In [132]:
#calculations
import numpy as np
import pandas as pd
import math
#scaling,metrics and tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf


In [133]:
#building LSTM models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [134]:
#visualisation and prediction
from bokeh.plotting import figure
from bokeh.io import output_notebook,show,push_notebook
import numpy as np
from bokeh.models import Legend

In [135]:
tata_stock=pd.read_csv('//content//NSE-TATAGLOBAL.csv')
tata_stock.head()

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55


In [136]:
tata_stock['Turnover (Lacs)'].value_counts()

2140.22    2
2195.84    2
637.58     2
1047.42    2
1646.87    2
          ..
1461.84    1
1841.84    1
1714.30    1
1896.92    1
803.56     1
Name: Turnover (Lacs), Length: 2030, dtype: int64

In [137]:
tata_stock1 =tata_stock['Last']
tata_stock1

0       233.50
1       233.80
2       235.00
3       236.25
4       234.00
         ...  
2030    118.80
2031    117.10
2032    120.35
2033    120.75
2034    121.10
Name: Last, Length: 2035, dtype: float64

In [138]:
#scaling the features ranging from one to two
scaler=MinMaxScaler(feature_range=(1,2))
tata_stock1=scaler.fit_transform(np.array(tata_stock1).reshape(-1,1))

In [139]:
tata_stock1

array([[1.62257604],
       [1.62380078],
       [1.62869973],
       ...,
       [1.16064503],
       [1.16227802],
       [1.16370688]])

In [13]:
out=[] #store thsoe in a dummy variable
for i in tata_stock1:
  out.append(i[0])
tata_stock1=out

In [140]:
tata_stock1

array([[1.62257604],
       [1.62380078],
       [1.62869973],
       ...,
       [1.16064503],
       [1.16227802],
       [1.16370688]])

In [141]:
#splitting teh dataset into 60-40
train_size=int(len(tata_stock1)*0.6)
test_size=len(tata_stock1)-train_size
train,test =tata_stock1[0:train_size],tata_stock1[train_size:len(tata_stock1)]
print(len(train),len(test))

1221 814


In [142]:
#convert an array of values into a dataset matrix
def create_dataset(dataset,look_back):
  dataX,dataY=[],[]
  for i in range(len(dataset)-look_back-1):
    a=dataset[i:(i+look_back)]
    dataX.append(a)
    dataY.append(dataset[i+look_back])
  return np.array(dataX),np.array(dataY)

In [171]:
look_back=3
trainX,trainY =create_dataset(train,look_back)
testX,testY = create_dataset(test,look_back)

In [172]:
trainX.shape

(1217, 3, 1)

In [173]:
trainY.shape

(1217, 1)

In [174]:
testX.shape

(810, 3, 1)

In [175]:
testY.shape

(810, 1)

In [176]:
testY

array([[1.31067565],
       [1.31965707],
       [1.32169831],
       [1.32373954],
       [1.30332721],
       [1.30536844],
       [1.30679731],
       [1.29332517],
       [1.30169422],
       [1.28250663],
       [1.25515411],
       [1.26883037],
       [1.26944274],
       [1.26474791],
       [1.25637885],
       [1.25637885],
       [1.27760768],
       [1.28495611],
       [1.27352521],
       [1.27311696],
       [1.28189426],
       [1.28985507],
       [1.28373137],
       [1.28536436],
       [1.26944274],
       [1.26883037],
       [1.27311696],
       [1.25556236],
       [1.24821392],
       [1.25719535],
       [1.23759951],
       [1.22902633],
       [1.2269851 ],
       [1.2453562 ],
       [1.26005307],
       [1.26842213],
       [1.26740151],
       [1.2506634 ],
       [1.24331496],
       [1.24494795],
       [1.24106961],
       [1.27352521],
       [1.2690345 ],
       [1.26229843],
       [1.26597265],
       [1.24923454],
       [1.23167993],
       [1.254

In [177]:
#reshape input to be [samples,time steps,features]
trainX=np.reshape(trainX,(trainX.shape[0],1,trainX.shape[1]))
testX=np.reshape(testX,(testX.shape[0],1,testX.shape[1]))

In [178]:
trainX.shape

(1217, 1, 3)

**Building LSTM model**

In [179]:
model= Sequential()
model.add(LSTM(4,return_sequences=True,input_shape=(1,look_back)))#four hidden layers
model.add(LSTM(4)) #second LSTM layer (stacked)
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.fit(trainX,trainY,epochs=100,batch_size=256,verbose=1) #256

Epoch 1/100
5/5 [==============================] - 6s 7ms/step - loss: 1.5883
Epoch 2/100
5/5 [==============================] - 0s 7ms/step - loss: 1.5455
Epoch 3/100
5/5 [==============================] - 0s 7ms/step - loss: 1.5018
Epoch 4/100
5/5 [==============================] - 0s 6ms/step - loss: 1.4572
Epoch 5/100
5/5 [==============================] - 0s 7ms/step - loss: 1.4115
Epoch 6/100
5/5 [==============================] - 0s 7ms/step - loss: 1.3649
Epoch 7/100
5/5 [==============================] - 0s 7ms/step - loss: 1.3171
Epoch 8/100
5/5 [==============================] - 0s 6ms/step - loss: 1.2681
Epoch 9/100
5/5 [==============================] - 0s 8ms/step - loss: 1.2179
Epoch 10/100
5/5 [==============================] - 0s 10ms/step - loss: 1.1668
Epoch 11/100
5/5 [==============================] - 0s 6ms/step - loss: 1.1146
Epoch 12/100
5/5 [==============================] - 0s 6ms/step - loss: 1.0613
Epoch 13/100
5/5 [==============================] - 0s 7ms/s

**Predicting the model**


In [180]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY.reshape(-1, 1))  # Reshape trainY to a 2D array
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY.reshape(-1, 1))  # Reshape testY to a 2D array


26/26 [==============================] - 0s 3ms/step


**Finding Out the RMSE value**

In [181]:
trainScore =math.sqrt(mean_squared_error(trainY,trainPredict[:, 0]))
print('Train Score: %0.2f RMSE' % trainScore)
testScore =math.sqrt(mean_squared_error(testY,testPredict[:, 0]))
print('Test Score: %0.2f RMSE' % testScore)

Train Score: 33.17 RMSE
Test Score: 31.13 RMSE


In [182]:
#shift train predictions for plotting
trainPredictPlot = np.zeros(len(tata_stock1))
trainPredictPlot[:]=np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back]=trainPredict[:,0]

In [183]:
#shift test predictions for plotting
testPredictPlot = np.zeros(len(tata_stock1),dtype=np.float64)
testPredictPlot[:]=np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(tata_stock1)-1]=testPredict[:,0]

In [184]:
from bokeh.plotting import figure, show, output_notebook
import pandas as pd
import numpy as np

output_notebook()

# Create the figure
p = figure(width=700, height=300)

# Plot actual stock prices
p.line(np.arange(len(tata_stock['Last'])), tata_stock['Last'], legend_label='Actual', color='black')

# Create DataFrames for train and test predictions with appropriate dates as the index
train_dates = tata_stock.index[:len(trainPredict)]
test_dates = tata_stock.index[-len(testPredict):]

tpp = pd.DataFrame(trainPredict, columns=['Close'], index=train_dates)
qpp = pd.DataFrame(testPredict, columns=['Close'], index=test_dates)

# Plot training predictions
p.line(np.arange(len(tpp)), tpp.Close, legend_label='Training', color='blue')

# Plot testing predictions
p.line(np.arange(len(tpp), len(tpp) + len(qpp)), qpp.Close, legend_label='Testing', color='orange')

# Set legend location
p.legend.location = 'top_right'

# Show the plot
show(p)
